# Projeto 2 - CDados

### Autores: Diego Guimarães Nuñez, Felipe Bakowski Nantes de Souza, Gustavo de Lima Mota

### Tema: Conseguimos prever o número de homicídios da cidade de São Paulo por ano?

### Features: PIB de São Paulo, Índice do Custo de Vida (ICV), Escolaridade, Tráfico de Drogas e Desemprego.

In [ ]:
import basedosdados as bd

In [4]:
# Para carregar o dado direto no pandas
desemprego = bd.read_table(dataset_id='br_me_caged',
table_id='dicionario',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 5324/5324 [00:00<00:00, 6188.64rows/s]


In [5]:
desemprego

,id_tabela,coluna,chave,cobertura_temporal,valor
0,microdados_estabelecimentos,fonte_desligamento,1,2020(1)2021,Dado Original/Sem Imputação
1,microdados_estabelecimentos,fonte_desligamento,2,2020(1)2021,Desligamentos Imputados Do CAGED
2,microdados_estabelecimentos,fonte_desligamento,3,2020(1)2021,Desligamentos Imputados Do Empregadorweb
3,microdados_estabelecimentos,tipo_empregador,0,2020(1)2021,CNPJ Raiz
4,microdados_estabelecimentos,tipo_empregador,2,2020(1)2021,CPF
...,...,...,...,...,...
5319,microdados_antigos_ajustes,regiao_corede,18,2007(1)2019,Sul
5320,microdados_antigos_ajustes,regiao_corede,19,2007(1)2019,Vale Do Cai
5321,microdados_antigos_ajustes,regiao_corede,20,2007(1)2019,Vl Rio Pardo
5322,microdados_antigos_ajustes,regiao_corede,21,2007(1)2019,Vl Rio Sinos


In [6]:
inep = bd.read_table(dataset_id='br_inep_saeb',
table_id='municipio',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 1726300/1726300 [06:42<00:00, 4291.16rows/s]


In [7]:
inep

,ano,rede,localizacao,sigla_uf,id_municipio,disciplina,serie,media,nivel_0,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,nivel_6,nivel_7,nivel_8,nivel_9,nivel_10
0,2011,Total - Estadual e Municipal,Rural,MA,2102507,MT,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,"Total - Federal, Estadual e Municipal",Rural,MS,5003504,LP,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,Total - Estadual e Municipal,Urbana,PA,1504109,MT,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,Privada,Total,SP,3556800,MT,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,"Total - Federal, Estadual, Municipal e Privada",Rural,BA,2901205,MT,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726295,2021,Total - Estadual e Municipal,Total,AP,1600550,MT,5,167.55,13.74,19.82,26.96,20.86,10.97,5.34,2.30,0.00,0.0,0.00,0.00
1726296,2021,"Total - Federal, Estadual e Municipal",Total,SE,2805307,MT,9,230.15,20.88,22.39,27.96,15.46,11.67,0.79,0.85,0.00,0.0,0.00,NaN
1726297,2021,Municipal,Total,PA,1500958,MT,5,176.34,5.18,20.34,27.96,23.64,12.94,6.72,1.34,1.56,0.0,0.32,0.00
1726298,2021,Total - Estadual e Municipal,Total,PI,2202251,LP,5,162.14,13.00,29.72,31.96,10.75,6.13,4.22,0.00,4.22,0.0,0.00,NaN
